Берем результаты других ноутбуков и объединяем их тут

In [25]:
%cd ..

/workspace/TabDDPM_copy/TabDDPM


/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [26]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ohe_noise.tabddpm_ohe_noise import *

from tqdm.notebook import tqdm

from os import listdir
from os.path import isfile, join, isdir

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

[KeOps] Compiling cuda jit compiler engine ... OK
[pyKeOps] Compiling nvrtc binder for python ... OK


In [27]:
datename = 'adult'

In [28]:
path = f'./eval/total/'
dirs = [join(path, f) for f in listdir(path) if isdir(join(path, f))]

In [29]:
results = []
for d in dirs:
    if datename in d:
        res = [pd.read_csv(join(d, f), header=[0, 1], skipinitialspace=True) for f in listdir(d) if isfile(join(d, f)) and 'csv' in f]
        results.extend(res)

results = pd.concat(results)
results = results.rename(columns={'Unnamed: 0_level_0':'', 
                                  'Unnamed: 1_level_0':'', 
                                  'Unnamed: 2_level_0':''}).drop(columns=[('base_metrics', '0')])

In [52]:
import numpy as np
import json

with open('/workspace/TabDDPM_copy/TabDDPM/Model_names_simple.json', 'r') as openfile:
    naming = json.load(openfile)

def get_positions(s, substring):
    positions = []
    start = 0
    while True:
        start = s.find(substring, start)
        if start == -1:
            break
        positions.append(start)
        start += len(substring)
    return positions
    
def get_simple_name(model_name):
    try:
        pos = get_positions(model_name, '_')[-2]
        model_name_short = model_name[pos+1:]

        if naming.get(model_name_short):
            return naming[model_name_short]
        else:
            pos = get_positions(model_name, '_')[-3]
            model_name_short = model_name[pos+1:]
            return naming[model_name_short]
    except:
        return 'TabDDPM'

results[('', 'Model name')] = results[('', 'Type')].apply(lambda x: get_simple_name(x))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

results[[
    ('', 'Model name') ,
    ('similarity', 'Column Shapes Score, %') ,
    ('similarity', 'Column Pair Trends Score, %') ,
    ('similarity', 'Overall Score (Average), %') ,
    ('similarity', 'Error rate (%) of column-wise density estimation, %') ,
    ('similarity', 'Error rate (%) of column-wise density estimation std, %') ,
    ('similarity', 'Error rate (%) of pair-wise column correlation score, %') ,
    ('similarity', 'Error rate (%) of pair-wise column correlation score std, %') ,
    ('mle', 'ROC - AUC обучения на синтетических данных') ,
    ('mle', 'ROC - AUC обучения на синтетических данных, std') ,
    ('detection', 'Score') ,
    ('DCR', 'Score') ,
    ('quality', 'alpha precision') ,
    ('quality', 'beta recall') ,
]].sort_values(('similarity', 'Overall Score (Average), %'), ascending=False).reset_index(drop=True)

similarity                              \
    Model name Column Shapes Score, % Column Pair Trends Score, %   
0      TabDDPM              98.912810                   97.958709   
1     Linear 2              98.707042                   97.354780   
2      Steps 5              98.654218                   97.271363   
3   Constant 2              98.566178                   97.288721   
4   Constant 3              98.528915                   97.202097   
5     Cosine 3              98.469334                   97.045242   
6   Constant 5              98.439237                   96.898567   
7   Constant 4              98.482233                   96.807447   
8   Constant 1              98.324785                   96.793173   
9     Cosine 2              98.324376                   96.791369   
10    Linear 1              98.175117                   96.606862   
11     Steps 3              98.076636                   96.531699   
12     Steps 2              98.183102                   96.289764   
13    Cosine 1              98.140106                   96.272589   
14  Constant 6              98.107961                   95.801826   
15     Steps 4              97.848551                   95.931102   
16     Steps 1              97.428212                   94.781710   

                               \
   Overall Score (Average), %   
0                   98.435759   
1                   98.030911   
2                   97.962791   
3                   97.927450   
4                   97.865506   
5                   97.757288   
6                   97.668902   
7                   97.644840   
8                   97.558979   
9                   97.557872   
10                  97.390990   
11                  97.304167   
12                  97.236433   
13                  97.206348   
14                  96.954894   
15                  96.889826   
16                  96.104961   

                                                        \
   Error rate (%) of column-wise density estimation, %   
0                                             1.087190   
1                                             1.292958   
2                                             1.345782   
3                                             1.433822   
4                                             1.471085   
5                                             1.530666   
6                                             1.560763   
7                                             1.517767   
8                                             1.675215   
9                                             1.675624   
10                                            1.824883   
11                                            1.923364   
12                                            1.816898   
13                                            1.859894   
14                                            1.892039   
15                                            2.151449   
16                                            2.571788   

                                                            \
   Error rate (%) of column-wise density estimation std, %   
0                                                 0.795566   
1                                                 0.768404   
2                                                 0.856513   
3                                                 0.737328   
4                                                 0.837941   
5                                                 0.841628   
6                                                 1.102585   
7                                                 1.054445   
8                                                 0.983070   
9                                                 1.035047   
10                                                1.131902   
11                                                1.528887   
12                                                1.035531   
13                                                1.00

similarity                              \
    Model name Column Shapes Score, % Column Pair Trends Score, %   
0      TabDDPM              98.912810                   97.958709   
1     Linear 2              98.707042                   97.354780   
2      Steps 5              98.654218                   97.271363   
3   Constant 2              98.566178                   97.288721   
4   Constant 3              98.528915                   97.202097   
5     Cosine 3              98.469334                   97.045242   
6   Constant 5              98.439237                   96.898567   
7   Constant 4              98.482233                   96.807447   
8   Constant 1              98.324785                   96.793173   
9     Cosine 2              98.324376                   96.791369   
10    Linear 1              98.175117                   96.606862   
11     Steps 3              98.076636                   96.531699   
12     Steps 2              98.183102                   96.289764   
13    Cosine 1              98.140106                   96.272589   
14  Constant 6              98.107961                   95.801826   
15     Steps 4              97.848551                   95.931102   
16     Steps 1              97.428212                   94.781710   

                               \
   Overall Score (Average), %   
0                   98.435759   
1                   98.030911   
2                   97.962791   
3                   97.927450   
4                   97.865506   
5                   97.757288   
6                   97.668902   
7                   97.644840   
8                   97.558979   
9                   97.557872   
10                  97.390990   
11                  97.304167   
12                  97.236433   
13                  97.206348   
14                  96.954894   
15                  96.889826   
16                  96.104961   

                                                        \
   Error rate (%) of column-wise density estimation, %   
0                                             1.087190   
1                                             1.292958   
2                                             1.345782   
3                                             1.433822   
4                                             1.471085   
5                                             1.530666   
6                                             1.560763   
7                                             1.517767   
8                                             1.675215   
9                                             1.675624   
10                                            1.824883   
11                                            1.923364   
12                                            1.816898   
13                                            1.859894   
14                                            1.892039   
15                                            2.151449   
16                                            2.571788   

                                                            \
   Error rate (%) of column-wise density estimation std, %   
0                                                 0.795566   
1                                                 0.768404   
2                                                 0.856513   
3                                                 0.737328   
4                                                 0.837941   
5                                                 0.841628   
6                                                 1.102585   
7                                                 1.054445   
8                                                 0.983070   
9                                                 1.035047   
10                                                1.131902   
11                                                1.528887   
12                                                1.035531   
13                                                1.00

In [38]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
results.sort_values(('similarity', 'Overall Score (Average), %'), ascending=False).reset_index(drop=True)

\
              Model                              Type   Data   
0           TabDDPM                           tabddpm  adult   
1   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_6  adult   
2   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_4  adult   
3   tabddpm_ON_QnSC  tabddpm_ON_QnSC_200k_const_0_001  adult   
4   tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_01  adult   
5   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_9  adult   
6   tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_25  adult   
7   tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_1  adult   
8   tabddpm_ON_QnSC      tabddpm_ON_QnSC_200k_const_0  adult   
9   tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_8  adult   
10  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_5  adult   
11  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_2  adult   
12  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_1  adult   
13  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_7  adult   
14  tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_5  adult   
15  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_3  adult   
16  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_0  adult   

        base_metrics                                                  \
   Original Logistic Synthetic Logistic Original Tree Synthetic Tree   
0           0.657095           0.655940      0.659913       0.637144   
1                NaN                NaN           NaN            NaN   
2           0.657095           0.653838      0.660537       0.639583   
3           0.657095           0.654271      0.660537       0.641565   
4           0.657095           0.653238      0.660537       0.641947   
5           0.657095           0.653194      0.660537       0.637206   
6           0.657095           0.656927      0.660537       0.626794   
7           0.657095           0.644242      0.660537       0.636185   
8                NaN                NaN           NaN            NaN   
9           0.657095           0.649962      0.660537       0.632960   
10               NaN                NaN           NaN            NaN   
11          0.657095           0.650767      0.660537       0.633810   
12          0.657095           0.650151      0.660537       0.619873   
13          0.657095           0.653799      0.660537       0.622244   
14          0.657095           0.619646      0.660537       0.617256   
15          0.657095           0.651843      0.660537       0.620571   
16          0.657095           0.654412      0.660537       0.628629   

                                                               similarity  \
   Accuracy Loss Logistic, % Accuracy Loss Tree, % Column Shapes Score, %   
0                   0.175836              3.450320              98.912810   
1                        NaN                   NaN              98.707042   
2                   0.495687              3.172278              98.654218   
3                   0.429740              2.872237              98.566178   
4                   0.587045              2.814451              98.528915   
5                   0.593765              3.532089              98.469334   
6                   0.025664              5.108483              98.439237   
7                   1.956116              3.686731              98.482233   
8                        NaN                   NaN              98.324785   
9                   1.085504              4.175032              98.324376   
10                       NaN                   NaN              98.175117   
11                  0.963002              4.046255              98.076636   
12                  1.056807              6.156297              98.183102   
13                  0.501661              5.797221              98.140106   
14                  5.699211              6.552487              98.107961   
15                  0.799326              6.050511              97.848551   
16                  0.408372              4.830576              97.428212   

            

In [7]:
results.sort_values(('similarity', 'Overall Score (Average), %'), ascending=False
                   ).reset_index(drop=True).to_excel(f'./_{datename}_notebooks/Result_united.xlsx')

In [5]:
naming

{'const_0': 'Constant 1',
 'const_0_001': 'Constant 2',
 'const_0_01': 'Constant 3',
 'const_0_1': 'Constant 4',
 'const_0_25': 'Constant 5',
 'const_0_5': 'Constant 6',
 'mult_0': 'Steps 1',
 'mult_1': 'Steps 2',
 'mult_2': 'Steps 3',
 'mult_3': 'Steps 4',
 'mult_4': 'Steps 5',
 'mult_5': 'Linear 1',
 'mult_6': 'Linear 2',
 'mult_7': 'Cosine 1',
 'mult_8': 'Cosine 2',
 'mult_9': 'Cosine 3'}